In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf
sess = tf.Session()

In [ ]:
from keras import backend as K
K.set_session(sess)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping

## Introduction

This tutorial was taken from the Keras blog: [Power image classifier using very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html). We've reimplemented part of it here for you to explore with

## Fetching data

The data is stored in `~/data/catsdogs/`.

In [ ]:
datagen = ImageDataGenerator(rotation_range=40, 
                             width_shift_range=.2, height_shift_range=.2, 
                             shear_range=.2, zoom_range=.2, 
                             fill_mode='nearest', horizontal_flip=True)

## Building the model

In [ ]:
def reluConv2d(x, filters=32):
    return Conv2D(filters=filters, kernel_size=(3,3), 
                  activation='relu', padding='same')(x)

def mp2d(x):
    return MaxPooling2D(pool_size=(2,2), 
                        padding='same')(x)

def convMP(x, filters=32):
    return mp2d(reluConv2d(x, filters))

def conv2dense(x, filters=64, dropout=None):
    x = Flatten()(x)
    x = Dense(filters, activation='relu')(x)
    if dropout is None:
        return x
    else:
        return Dropout(dropout)(x)

In [ ]:
input_img = Input(shape=(150,150,3))
x = convMP(input_img)
x = convMP(x)
x = convMP(x, 64)
x = conv2dense(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(input_img, output, name='CatDogClf')

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## Training the beast

In [ ]:
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,
                                   shear_range=0.2,zoom_range=0.2)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        '/home/asberk/data/catsdogs/train/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

In [ ]:
validation_generator = test_datagen.flow_from_directory(
        '/home/asberk/data/catsdogs/validation/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=2002 // batch_size,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps=800 // batch_size)

In [ ]:
model.save('CatDogClf.h5')

## Auto-encoders

Check out [this tutorial](https://blog.keras.io/building-autoencoders-in-keras.html) if you're interested in auto-encoders.

## Text-processing stuff

If you're bored with everything above, come hang out with me during the problem solving session and we can hack on some machine learning and `nltk` stuff using the UBC Library Open Data API.